# 確率ロボティクス2017第11回

上田隆一

2017年11月29日@千葉工業大学

## 今日やること

* POMDP

## POMDP

* 部分観測マルコフ決定過程
* 状態が自明でない場合のマルコフ決定過程
  * センサ情報から得られるが、部分的
* ほぼ、自律型ロボットの問題と等価
  * 状態が分かっているものをロボットと言うかどうか
* 他者がいるとさらに問題が難しくなるが、POMDPでは考えない/扱えない


## 定式化

* 状態が直接観測できず、観測から間接的に分かる
  * 要は自己位置推定の問題に行動決定をくっつけたもの
* 各シンボルの定義
  * 時刻: $\mathcal{T} = \{t | t=0,1,2,\dots,T\}$
  * 状態: $\mathcal{S} = \{s_i | i=1,2,3,\dots,N\}$
    * どんな状態があるかはknown、どの状態にいるかはunknown
  * 行動: $\mathcal{A} = \{a_j | j=1,2,3,\dots,M\}$
  * 状態遷移: $\mathcal{P}_{ss'}^a$
  * 報酬: $\mathcal{R}_{ss'}^a$
  * 観測: $\mathcal{O}$スイカ割り

## 問題の例
 
* スイカ割り
  * 自分とスイカの相対姿勢がはっきり分からない
  * 他者からの情報で
* 真っ暗なビルから脱出
  * 壁に手をついて自己位置推定
* 実はスイカ割りで目隠しを取っても、ビルに照明がついていても、我々は正確な位置計測ができている訳ではない
  * 結局POMDP


## 自己位置推定の曖昧さ

* 例えばパーティクルフィルタでもカルマンフィルタでも分布は常に広がりを持つ
* 広がっているときに平均値（と決定論的方策$\pi$）を信じて動くとどうなるか？
* そもそも平均値が意味を持たないこともある
* 平均値でなくてもっといい行動の選び方はないだろうか？

<img width=300 src="./particle_mean.png">

## 入力が信念の方策

* エージェントが$bel$から行動決定
* 状態空間の上で定義された方策ではなく、$bel$が属する空間の上で定義された方策となる
  * 「信念に対する方策」「信念方策」
* どうやって求めるか？
* そもそも求められるのか？


## 問題の大きさ

* 状態が100個、行動が2種類の場合
* 有限MDP
  * 方策のパターンは$2^{100}$通り
  * 価値反復を使うと
    * $O$(状態数・行動数・タスクの長さ)で方策を計算可能
* POMDP
  * $bel$は100個の状態上で定義される関数
    * 無限に存在
  * 無限にあるものに対する方策のパターンは無限


# POMDPとハンドコーディング

* 例えば、パーティクルの分布から移動ロボットの行動を決めるコードを書いてみましょう
  * 例えば脱輪したり縁石に乗り上げたりしないで、できるだけ早く角を曲がる場合を考えてみましょう
  * （たいていの場合）ほぼ無限に場合分けできる
    * 疲れる
    * そもそも無理？
* 豊富な研究テーマ

<img width="300" src="corner-300x110.png" />

## 対策

* センサを強化
* 観測戦略
  * 位置の情報が得られそうな行動を能動的にとる
* センサやアクチュエータの配置、ロボットの形状の工夫（身体性）
  * ぶつかっても壊れない
  * 情報を得やすい位置にセンサをつける
  * 情報を得やすい/情報がなくても動けるようにアクチュエータを選定して取り付け
* 計算でなんとか良い行動を見つける

## 計算でなんとか
* だいたい以下のような方針になる
* 方法1:
  * $bel$の形状一個一個を状態と皆して有限MDPのように問題を解く
* 方法2:
  * 事前に状態が既知の有限MDPの問題を解いて$bel$からなんとか行動を得る
  * 分布の平均値やその他代表値を1つ選んで行動決定するのもこの方法の単純な実装

## 方法1（AMDPと呼ばれる手法群）

* 信念の数を有限個に近似
* 例[Roy 99]
  * 距離センサを持つ移動ロボットのナビゲーション問題
  * 4次元の状態空間を作る
    * $xy\theta$を離散化
    * $bel$がどれだけ曖昧か数値化して離散化
    * 状態遷移はなんとか計算
  * あとは価値反復等で計算
  * 得られる行動
    * <span style="color:red">自己位置が分からなくならないように壁沿いを走る</span>

## 方法2（$Q_\text{MDP}$, 他）

* 有限MDPの計算結果を利用する
  * 状態遷移の法則性が分かっているが、ロボットが自分の状態を完全に分からない場合に使える
* 例[Littman95]
  * 確率密度関数$bel$と価値関数$V$から価値の期待値を計算

## $Q_\text{MDP}$の例

* 例1: 実際のロボットの動き: https://youtu.be/fsQicKXE5AU
* 例2: 碁盤の目の環境
  * 1マスが1状態
  * 行動は上下左右
  * 同じ重みを持ったパーティクル10個が分布
  * タスクはゴール（G）に最短歩数で到達すること
  * 数字はコスト
  * 灰色のところに入ろうとすると戻される
* 問題1: 一番「価値の高い」行動は？

<img width="300" src="qmdp.png" />

* 問題2: この環境だとどうなる？

<img width="300" src="qmpd2.png" />

## 最近やっている研究

* probabilistic flow control[Ueda 2015]
  * 期待値計算において「重み = パーティクルの重み/価値」とする
    * ゴールに近いパーティクルが行動決定に大きな影響を与える
    * 投機的な行動が生成され、ロボットがゴールを探すようになる
* ただしこれでもデッドロックは発生
  * 研究は続く・・・
  
<img width="300" src="pfc.png" />


## PFCで動くロボット

* 次のような環境
  * 距離センサでロボットが自己位置推定
  <img width="300" src="env_pfc.png" />
  * 自己位置推定が常に次のような状態
   <img width="300" src="pfc_robot-300x292.png" />
    * ほとんど推定できていない

* ロボットがゴールに吸い込まれる
  * ランダムウォークだと止まったり同じところを行ったり来たりする
  * https://youtu.be/qs7JUygUzyI
